In [ ]:
! pip install pylatexenc

     |████████████████████████████████| 162 kB 5.2 MB/s 
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136835 sha256=15a3e2dc876a57140cc343f79e10d5e410c55fc484cc0d2a6c8f576eaf73b32b
  Stored in directory: /root/.cache/pip/wheels/f1/8a/f5/33ee79d4473eb201b519fa40f989b842e373237395a3421f52
Successfully built pylatexenc


In [ ]:
! pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

  Cloning https://github.com/qiskit-community/qiskit-textbook.git to /tmp/pip-req-build-uxbndjr8
  Running command git clone -q https://github.com/qiskit-community/qiskit-textbook.git /tmp/pip-req-build-uxbndjr8
     |████████████████████████████████| 6.5 MB 5.4 MB/s 
     |████████████████████████████████| 18.0 MB 293 kB/s 
     |████████████████████████████████| 240 kB 26.8 MB/s 
     |████████████████████████████████| 198 kB 45.2 MB/s 
     |████████████████████████████████| 112 kB 13.0 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 1.6 MB 42.2 MB/s 
     |████████████████████████████████| 943 kB 45.4 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 4.0 MB 37.1 MB/s 
     |████████████████████████████████| 1

In [ ]:
from qiskit.providers.aer import AerSimulator
from qiskit import *

#### ARX rounds

##### Addition utility function

In [ ]:
def add(n):
  a = QuantumRegister(n,'a')
  b = QuantumRegister(n,'b')
  ancilla = QuantumRegister(1, 'ancilla')
  z = QuantumRegister(1, 'z')
  qc = QuantumCircuit (a, b, ancilla, z, name="add")
  # qc = QuantumCircuit (a, b, ancilla, name="add")
  # qc.draw(output='mpl')

  for i in range(1,n):
    qc.cx(a[i],b[i])

  qc.cx(a[1], ancilla)
  qc.ccx(a[0], b[0], ancilla)
  qc.cx(a[2], a[1])
  qc.ccx(ancilla, b[1], a[1])
  qc.cx(a[3], a[2])

  for i in range(2,n-2):        
    qc.ccx(a[i-1],b[i],a[i])
    qc.cx(a[i+2],a[i+1])

  qc.ccx(a[n-3],b[n-2],a[n-2])
  qc.cx(a[n-1],z)

  qc.ccx(a[n-2],b[n-1],z)
  for i in range(1, n-1):
    qc.x(b[i])

  qc.cx(ancilla, b[1])
  for i in range(2, n):
    qc.cx(a[i-1],b[i])

  qc.ccx(a[n-3],b[n-2],a[n-2])
  for i in range(n-3,1,-1):
    qc.ccx(a[i-1],b[i],a[i])
    qc.cx(a[i+2],a[i+1])
    qc.x(b[i+1])

  qc.ccx(ancilla,b[1],a[1])
  qc.cx(a[3],a[2])
  qc.x(b[2])

  qc.ccx(a[0],b[0],ancilla)
  qc.cx(a[2],a[1])
  qc.x(b[1])

  qc.cx(a[1],ancilla)

  for i in range(0,n):
    qc.cx(a[i],b[i])

  # qc.measure_all()
  # qc.draw(output='mpl', filename="add_"+str(n))
  return qc

In [ ]:
def additionMod_2n(n):           # a,b..........a,b=a+b
  a = QuantumRegister(n,'a')
  b = QuantumRegister(n,'b')
  ancilla = QuantumRegister(1, 'ancilla')
  qc = QuantumCircuit (a, b, ancilla,  name="add_mod_32")

  l=list(range(n*2+1))
  l.remove(n-1)
  l.remove(n*2-1)

  add_circuit=add(n-1).to_gate()
  # print("additionMod32")
  qc.append(add_circuit, l+[n*2-1])
  qc.cx(a[n-1],b[n-1])

  return qc

##### XOR utility function

In [ ]:
def XOR(n):             # a,b ............ a=a^b,b
  a=QuantumRegister(n,'a')
  b=QuantumRegister(n,'b')
  qc=QuantumCircuit(a,b, name="XOR")
  for i in range(n):
    qc.cx(b[i],a[i])

  # qc.draw(output='mpl', filename='XOR')
  
  return qc

##### Rotation utility

In [ ]:
def swap_n(n):          # n is size of each register

  a=QuantumRegister(n,'a')
  b=QuantumRegister(n,'b')
  qc=QuantumCircuit(a,b,name="swap_"+str(n))

  for i in range(n):
    qc.swap(a[i],b[i])

  # qc.draw(output='mpl', filename="swap_"+str(n))

  return qc

In [ ]:
# def n_rotateBy_d(n, d):

#     a=QuantumRegister(n,'a')
#     qc=QuantumCircuit(a, name=str(n)+"_rotateBy_"+str(d))

#     if(d == 0 or d == n):
#         return;
#     i = d
#     j = n - d
#     while (i != j):
         
#         if(i < j): # A is shorter
#             # swap(arr, d - i, d + j - i, i)
#             l=list(range(d-i,d))
#             m=list(range(d+j-i,d+j))
#             qc.append(swap_n(i).to_gate(), l+m)
#             j -= i
         
#         else: # B is shorter
#             # swap(arr, d - i, d, j)
#             l=list(range(d-i,d-i+j))
#             m=list(range(d,d+j))
#             qc.append(swap_n(j).to_gate(), l+m)
#             i -= j
     
#     # swap(arr, d - i, d, i)
#     l=list(range(d-i,d))
#     m=list(range(d,d+i))
#     qc.append(swap_n(i).to_gate(), l+m)

#     # print(arr)

#     return qc

In [ ]:
def n_rotateBy_d(n, d):

    a=QuantumRegister(n,'a')
    qc=QuantumCircuit(a, name=str(n)+"_rotateBy_"+str(d))

    if(d == 0 or d == n):
        return;
    i = d
    j = n - d
    while (i != j):
         
        if(i < j): # A is shorter
            # swap(arr, d - i, d + j - i, i)
            l=list(range(n-(d),n-(d-i)))
            m=list(range(n-(d+j),n-(d+j-i)))
            qc.append(swap_n(i).to_gate(), l+m)
            j -= i
         
        else: # B is shorter
            # swap(arr, d - i, d, j)
            l=list(range(n-(d-i+j),n-(d-i)))
            m=list(range(n-(d+j),n-(d)))
            qc.append(swap_n(j).to_gate(), l+m)
            i -= j
     
    # swap(arr, d - i, d, i)
    l=list(range(n-(d),n-(d-i)))
    m=list(range(n-(d+i),n-(d)))
    qc.append(swap_n(i).to_gate(), l+m)

    # print(arr)

    return qc

In [ ]:
def swap(arr, fi, si, d):
    for i in range(d):
        temp = arr[fi + i];
        arr[fi + i] = arr[si + i];
        arr[si + i] = temp;

In [ ]:
def classical_n_rotateBy_d(arr,n,d):
  arr=arr[::-1]
  print(arr)
  if(d == 0 or d == n):
        return arr

  i = d
  j = n - d
  while (i != j):
        
      if(i < j): # A is shorter
          swap(arr, d - i, d + j - i, i)
          j -= i
        
      else: # B is shorter
          swap(arr, d - i, d, j)
          i -= j
    
  swap(arr, d - i, d, i)

  arr=arr[::-1]
  print(arr)
  return arr


#### one arx

In [ ]:
def one_arx(n, shift, d_l):
  w=QuantumRegister(n, 'w')
  x=QuantumRegister(n, 'x')
  y=QuantumRegister(n, 'y')
  z=QuantumRegister(n, 'z')
  ancilla = QuantumRegister(1, 'ancilla')

  qc=QuantumCircuit(w,x,y,z,ancilla,name="one arx")

  w_l = list(range(n*0,n*1))
  x_l = list(range(n*1,n*2))
  y_l = list(range(n*2,n*3))
  z_l = list(range(n*3,n*4))
  ancilla_l = [n*4]
  
  # print("w+=x: x_l",x_l,"w_l",w_l)
  qc.append(additionMod_2n(n).to_gate(), x_l+w_l+ancilla_l)        # a+=b   ---> w+=x
  # print("z^=w: z_l",z_l,"w_l",w_l)
  qc.append(XOR(n).to_gate(), z_l+w_l)                             # d^=a   ---> z^=w
  # qc.append(n_rotateBy_d(n, shift).to_gate(), z_l)                 # d<<<16 ---> z

  # print("d<<shift: d_l",d_l)
  # d_l = classical_n_rotateBy_d(d_l,n,shift)
  # print("After: d_l",d_l)


  return qc

#### initialize

In [ ]:
def init(row0, row1, row2, row3, base, n):
  A0,A1,A2,A3 = row0.split()
  B0,B1,B2,B3 = row1.split()
  C0,C1,C2,C3 = row2.split()
  D0,D1,D2,D3 = row3.split()

  A0=bin(int(A0, base))[2:].zfill(n)            # 16 for hexadecimal value
  B0=bin(int(B0, base))[2:].zfill(n)
  C0=bin(int(C0, base))[2:].zfill(n)
  D0=bin(int(D0, base))[2:].zfill(n)

  A1=bin(int(A1, base))[2:].zfill(n)
  B1=bin(int(B1, base))[2:].zfill(n)
  C1=bin(int(C1, base))[2:].zfill(n)
  D1=bin(int(D1, base))[2:].zfill(n)

  A2=bin(int(A2, base))[2:].zfill(n)
  B2=bin(int(B2, base))[2:].zfill(n)
  C2=bin(int(C2, base))[2:].zfill(n)
  D2=bin(int(D2, base))[2:].zfill(n)

  A3=bin(int(A3, base))[2:].zfill(n)
  B3=bin(int(B3, base))[2:].zfill(n)
  C3=bin(int(C3, base))[2:].zfill(n)
  D3=bin(int(D3, base))[2:].zfill(n)

  return A0,A1,A2,A3,B0,B1,B2,B3,C0,C1,C2,C3,D0,D1,D2,D3

#### measure

In [ ]:
def measure(qc, n, n_shots=1):#, i=0,j=0,k=0,l=0,A=0,B=0,C=0,D=0,ancilla=0):
  # Select the AerSimulator from the Aer provider
  simulator = AerSimulator(method='matrix_product_state')
  # Run and get counts, using the matrix_product_state method
  tcirc = transpile(qc, simulator)
  result = simulator.run(tcirc, shots=n_shots).result()
  counts = result.get_counts(0)
  print(counts)
  for key in counts.keys():

      # print(hex(int(key[-n:],2)), hex(int(key[-n*2:-n*1],2)), hex(int(key[-n*3:-n*2],2)), hex(int(key[-n*4:-n*3],2)) )
      # print(hex(int(key[-n*5:-n*4],2)), hex(int(key[-n*6:-n*5],2)), hex(int(key[-n*7:-n*6],2)), hex(int(key[-n*8:-n*7],2)) )
      # print(hex(int(key[-n*9:-n*8],2)), hex(int(key[-n*10:-n*9],2)), hex(int(key[-n*11:-n*10],2)), hex(int(key[-n*12:-n*11],2)))
      # print(hex(int(key[-n*13:-n*12],2)), hex(int(key[-n*14:-n*13],2)), hex(int(key[-n*15:-n*14],2)), hex(int(key[-n*16:-n*15],2)))
      # print()

      # print(key[-n:], key[-n*2:-n*1], key[-n*3:-n*2], key[-n*4:-n*3])
      # print(key[-n*5:-n*4], key[-n*6:-n*5], key[-n*7:-n*6], key[-n*8:-n*7])
      # print(key[-n*9:-n*8], key[-n*10:-n*9], key[-n*11:-n*10], key[-n*12:-n*11])
      # print(key[-n*13:-n*12], key[-n*14:-n*13], key[-n*15:-n*14], key[-n*16:-n*15])
      # print()

      # print("A0\t",key[-n:], "\t", hex(int(key[-n:],2)))
      # print("A1\t",key[-n*2:-n*1], "\t", hex(int(key[-n*2:-n*1],2)))
      # print("A2\t",key[-n*3:-n*2], "\t", hex(int(key[-n*3:-n*2],2)))
      # print("A3\t",key[-n*4:-n*3], "\t", hex(int(key[-n*4:-n*3],2)))


      # print("B0\t",key[-n*5:-n*4], "\t", hex(int(key[-n*5:-n*4],2)))
      # print("B1\t",key[-n*6:-n*5], "\t", hex(int(key[-n*6:-n*5],2)))
      # print("B2\t",key[-n*7:-n*6], "\t", hex(int(key[-n*7:-n*6],2)))
      # print("B3\t",key[-n*8:-n*7], "\t", hex(int(key[-n*8:-n*7],2)))

      # print("C0\t",key[-n*9:-n*8], "\t", hex(int(key[-n*9:-n*8],2)))
      # print("C1\t",key[-n*10:-n*9], "\t", hex(int(key[-n*10:-n*9],2)))
      # print("C2\t",key[-n*11:-n*10], "\t", hex(int(key[-n*11:-n*10],2)))
      # print("C3\t",key[-n*12:-n*11], "\t", hex(int(key[-n*12:-n*11],2)))

      # print("D0\t",key[-n*13:-n*12], "\t", hex(int(key[-n*13:-n*12],2)))
      # print("D1\t",key[-n*14:-n*13], "\t", hex(int(key[-n*14:-n*13],2)))
      # print("D2\t",key[-n*15:-n*14], "\t", hex(int(key[-n*15:-n*14],2)))
      # print("D3\t",key[-n*16:-n*15], "\t", hex(int(key[-n*16:-n*15],2)))


      x=key[::-1]
      print(a0_l, a1_l, a2_l, a3_l)
      print(b0_l, b1_l, b2_l, b3_l)
      print(c0_l, c1_l, c2_l, c3_l)
      print(d0_l, d1_l, d2_l, d3_l)
      print()

      print(hex(int(get_num(x,a0_l), 2)), hex(int(get_num(x,a1_l), 2)), hex(int(get_num(x,a2_l), 2)), hex(int(get_num(x,a3_l), 2)))
      print(hex(int(get_num(x,b0_l), 2)), hex(int(get_num(x,b1_l), 2)), hex(int(get_num(x,b2_l), 2)), hex(int(get_num(x,b3_l), 2)))
      print(hex(int(get_num(x,c0_l), 2)), hex(int(get_num(x,c1_l), 2)), hex(int(get_num(x,c2_l), 2)), hex(int(get_num(x,c3_l), 2)))
      print(hex(int(get_num(x,d0_l), 2)), hex(int(get_num(x,d1_l), 2)), hex(int(get_num(x,d2_l), 2)), hex(int(get_num(x,d3_l), 2)))
      print()

      print(get_num(x,a0_l), get_num(x,a1_l), get_num(x,a2_l), get_num(x,a3_l))
      print(get_num(x,b0_l), get_num(x,b1_l), get_num(x,b2_l), get_num(x,b3_l))
      print(get_num(x,c0_l), get_num(x,c1_l), get_num(x,c2_l), get_num(x,c3_l))
      print(get_num(x,d0_l), get_num(x,d1_l), get_num(x,d2_l), get_num(x,d3_l))

      print()
      print()

In [ ]:
exmaple = "1010101001011111"

In [ ]:
exmaple = exmaple[::-1]

In [ ]:
my_list = [0,1,2,3]

In [ ]:
num=""
for i in my_list:
  num+=exmaple[i]
num

'1111'

In [ ]:
def get_num(ori_num, my_list):
  num=""
  for i in my_list:
    num+=ori_num[i]
  num

  return num[::-1]

In [ ]:
get_num("1010101001011111"[::-1], [12,13,14,15])

'1010'

In [ ]:
# d0_l

In [ ]:
# a0_l

#### Take inputs

In [ ]:
# n=32

In [ ]:
# shift0 = 16
# shift1 = 8
# shift2 = 12
# shift3 = 7

In [ ]:
# row0 = "61707865 3320646e 79622d32 6b206574"
# row1 = "03020100 07060504 0b0a0908 0f0e0d0c"
# row2 = "13121110 17161514 1b1a1918 1f1e1d1c"
# row3 = "00000001 00000000 4a000000 00000000"

In [ ]:
n=5

In [ ]:
shift0 = 4
shift1 = 2
shift2 = 3
shift3 = 1

In [ ]:
row0 = "01011 00111 01110 10100"
row1 = "00101 01010 00001 11100"
row2 = "11111 01011 11100 10101"
row3 = "00101 00100 10101 10111"

In [ ]:
base = 2

In [ ]:
n=8

In [ ]:
shift0 = 4
shift1 = 3
shift2 = 2
shift3 = 1

In [ ]:
row0 = "00101011 00101111 01110111 10100101"
row1 = "01110101 01000010 00001000 11100111"
row2 = "11111111 01011111 11100011 10101111"
row3 = "00010101 00111100 10101111 10111111"

In [ ]:
base = 2

#### Circuit

In [ ]:
a0=QuantumRegister(n, 'a0')
a1=QuantumRegister(n, 'a1')
a2=QuantumRegister(n, 'a2')
a3=QuantumRegister(n, 'a3')

b0=QuantumRegister(n, 'b0')
b1=QuantumRegister(n, 'b1')
b2=QuantumRegister(n, 'b2')
b3=QuantumRegister(n, 'b3')

c0=QuantumRegister(n, 'c0')
c1=QuantumRegister(n, 'c1')
c2=QuantumRegister(n, 'c2')
c3=QuantumRegister(n, 'c3')

d0=QuantumRegister(n, 'd0')
d1=QuantumRegister(n, 'd1')
d2=QuantumRegister(n, 'd2')
d3=QuantumRegister(n, 'd3')

ancilla0 = QuantumRegister(1, 'ancilla0')
ancilla1 = QuantumRegister(1, 'ancilla1')
ancilla2 = QuantumRegister(1, 'ancilla2')
ancilla3 = QuantumRegister(1, 'ancilla3')

ori_a0=QuantumRegister(n, 'ori_a0')
ori_a1=QuantumRegister(n, 'ori_a1')
ori_a2=QuantumRegister(n, 'ori_a2')
ori_a3=QuantumRegister(n, 'ori_a3')

ori_b0=QuantumRegister(n, 'ori_b0')
ori_b1=QuantumRegister(n, 'ori_b1')
ori_b2=QuantumRegister(n, 'ori_b2')
ori_b3=QuantumRegister(n, 'ori_b3')

ori_c0=QuantumRegister(n, 'ori_c0')
ori_c1=QuantumRegister(n, 'ori_c1')
ori_c2=QuantumRegister(n, 'ori_c2')
ori_c3=QuantumRegister(n, 'ori_c3')

ori_d0=QuantumRegister(n, 'ori_d0')
ori_d1=QuantumRegister(n, 'ori_d1')
ori_d2=QuantumRegister(n, 'ori_d2')
ori_d3=QuantumRegister(n, 'ori_d3')

# cR = ClassicalRegister(n)
# cR2 = ClassicalRegister(2)

# extra = QuantumRegister(2, "extra")

qc=QuantumCircuit(a0,a1,a2,a3,b0,b1,b2,b3,c0,c1,c2,c3,d0,d1,d2,d3,ancilla0,ancilla1,ancilla2,ancilla3, ori_a0,ori_a1,ori_a2,ori_a3,ori_b0,ori_b1,ori_b2,ori_b3,ori_c0,ori_c1,ori_c2,ori_c3,ori_d0,ori_d1,ori_d2,ori_d3)


##### initialise states

In [ ]:
A0,A1,A2,A3,B0,B1,B2,B3,C0,C1,C2,C3,D0,D1,D2,D3 = init(row0, row1, row2, row3, base, n)

###### row 0

In [ ]:
A0=A0[::-1]
B0=B0[::-1]
C0=C0[::-1]
D0=D0[::-1]

for i in range(len(A0)):
  if A0[i]=='1':
    qc.x(a0[i])

for i in range(len(B0)):
  if B0[i]=='1':
    qc.x(b0[i])

for i in range(len(C0)):
  if C0[i]=='1':
    qc.x(c0[i])

for i in range(len(D0)):
  if D0[i]=='1':
    qc.x(d0[i])

# qc.barrier()

###### row 1

In [ ]:
A1=A1[::-1]
B1=B1[::-1]
C1=C1[::-1]
D1=D1[::-1]

for i in range(len(A1)):
  if A1[i]=='1':
    qc.x(a1[i])

for i in range(len(B1)):
  if B1[i]=='1':
    qc.x(b1[i])

for i in range(len(C1)):
  if C1[i]=='1':
    qc.x(c1[i])

for i in range(len(D1)):
  if D1[i]=='1':
    qc.x(d1[i])

# qc.barrier()

###### row 2

In [ ]:
A2=A2[::-1]
B2=B2[::-1]
C2=C2[::-1]
D2=D2[::-1]

for i in range(len(A2)):
  if A2[i]=='1':
    qc.x(a2[i])

for i in range(len(B2)):
  if B2[i]=='1':
    qc.x(b2[i])

for i in range(len(C2)):
  if C2[i]=='1':
    qc.x(c2[i])

for i in range(len(D2)):
  if D2[i]=='1':
    qc.x(d2[i])

# qc.barrier()

###### row 3

In [ ]:
A3=A3[::-1]
B3=B3[::-1]
C3=C3[::-1]
D3=D3[::-1]

for i in range(len(A3)):
  if A3[i]=='1':
    qc.x(a3[i])

for i in range(len(B3)):
  if B3[i]=='1':
    qc.x(b3[i])

for i in range(len(C3)):
  if C3[i]=='1':
    qc.x(c3[i])

for i in range(len(D3)):
  if D3[i]=='1':
    qc.x(d3[i])

# qc.barrier()

#### initialising lists

In [ ]:
a0_l=list(range(n*0,n*1))        # A0 block
b0_l=list(range(n*4,n*5))        # B0 block
c0_l=list(range(n*8,n*9))        # C0 block
d0_l=list(range(n*12,n*13))        # D0 block
ancilla0_l=list([n*16+0])

a1_l=list(range(n*1,n*2))        # A1 block
b1_l=list(range(n*5,n*6))        # B1 block
c1_l=list(range(n*9,n*10))        # C1 block
d1_l=list(range(n*13,n*14))        # D1 block
ancilla1_l=list([n*16+1])

a2_l=list(range(n*2,n*3))        # A2 block
b2_l=list(range(n*6,n*7))        # B2 block
c2_l=list(range(n*10,n*11))        # C2 block
d2_l=list(range(n*14,n*15))        # D2 block
ancilla2_l=list([n*16+2])

a3_l=list(range(n*3,n*4))        # A3 block
b3_l=list(range(n*7,n*8))        # B3 block
c3_l=list(range(n*11,n*12))        # C3 block
d3_l=list(range(n*15,n*16))        # D3 block
ancilla3_l=list([n*16+3])


temp=n*16+3+1


ori_a0_l=list(range(temp+n*0,temp+n*1))        # A0 block
ori_b0_l=list(range(temp+n*4,temp+n*5))        # B0 block
ori_c0_l=list(range(temp+n*8,temp+n*9))        # C0 block
ori_d0_l=list(range(temp+n*12,temp+n*13))        # D0 block

ori_a1_l=list(range(temp+n*1,temp+n*2))        # A1 block
ori_b1_l=list(range(temp+n*5,temp+n*6))        # B1 block
ori_c1_l=list(range(temp+n*9,temp+n*10))        # C1 block
ori_d1_l=list(range(temp+n*13,temp+n*14))        # D1 block

ori_a2_l=list(range(temp+n*2,temp+n*3))        # A2 block
ori_b2_l=list(range(temp+n*6,temp+n*7))        # B2 block
ori_c2_l=list(range(temp+n*10,temp+n*11))        # C2 block
ori_d2_l=list(range(temp+n*14,temp+n*15))        # D2 block

ori_a3_l=list(range(temp+n*3,temp+n*4))        # A3 block
ori_b3_l=list(range(temp+n*7,temp+n*8))        # B3 block
ori_c3_l=list(range(temp+n*11,temp+n*12))        # C3 block
ori_d3_l=list(range(temp+n*15,temp+n*16))        # D3 block

#### copying values to original

In [ ]:
for i in range(n):
  qc.cx(a0[i],ori_a0[i])
  qc.cx(b0[i],ori_b0[i])
  qc.cx(c0[i],ori_c0[i])
  qc.cx(d0[i],ori_d0[i])


  qc.cx(a1[i],ori_a1[i])
  qc.cx(b1[i],ori_b1[i])
  qc.cx(c1[i],ori_c1[i])
  qc.cx(d1[i],ori_d1[i])


  qc.cx(a2[i],ori_a2[i])
  qc.cx(b2[i],ori_b2[i])
  qc.cx(c2[i],ori_c2[i])
  qc.cx(d2[i],ori_d2[i])


  qc.cx(a3[i],ori_a3[i])
  qc.cx(b3[i],ori_b3[i])
  qc.cx(c3[i],ori_c3[i])
  qc.cx(d3[i],ori_d3[i])

#### applying arx

In [ ]:
qc1=qc.copy()

In [ ]:
# qc=qc1.copy()

In [ ]:
qc2=qc.copy()
qc2.measure_all()
measure(qc2,n)
  
for i in range(10):
  print(i, 'column round')
  print('column 0')
  qc.append(one_arx(n,shift0, d0_l), a0_l+b0_l+c0_l+d0_l+ancilla0_l)           # quarter round on column 0
  d0_l = classical_n_rotateBy_d(d0_l,n,shift0)
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  qc.append(one_arx(n,shift1, b0_l), c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  b0_l = classical_n_rotateBy_d(b0_l,n,shift1)
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  qc.append(one_arx(n,shift2, d0_l), a0_l+b0_l+c0_l+d0_l+ancilla0_l)
  d0_l = classical_n_rotateBy_d(d0_l,n,shift2)
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  qc.append(one_arx(n,shift3, b0_l), c0_l+d0_l+a0_l+b0_l+ancilla0_l)
  b0_l = classical_n_rotateBy_d(b0_l,n,shift3)
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  print('column 1')
  qc.append(one_arx(n,shift0, d1_l), a1_l+b1_l+c1_l+d1_l+ancilla1_l)            # quarter round on column 1
  d1_l = classical_n_rotateBy_d(d1_l,n,shift0)
  qc.append(one_arx(n,shift1, b1_l), c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  b1_l = classical_n_rotateBy_d(b1_l,n,shift1)
  qc.append(one_arx(n,shift2, d1_l), a1_l+b1_l+c1_l+d1_l+ancilla1_l)
  d1_l = classical_n_rotateBy_d(d1_l,n,shift2)
  qc.append(one_arx(n,shift3, b1_l), c1_l+d1_l+a1_l+b1_l+ancilla1_l)
  b1_l = classical_n_rotateBy_d(b1_l,n,shift3)

  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  print('column 2')
  qc.append(one_arx(n,shift0, d2_l), a2_l+b2_l+c2_l+d2_l+ancilla2_l)            # quarter round on column 2
  d2_l = classical_n_rotateBy_d(d2_l,n,shift0)
  qc.append(one_arx(n,shift1, b2_l), c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  b2_l = classical_n_rotateBy_d(b2_l,n,shift1)
  qc.append(one_arx(n,shift2, d2_l), a2_l+b2_l+c2_l+d2_l+ancilla2_l)
  d2_l = classical_n_rotateBy_d(d2_l,n,shift2)
  qc.append(one_arx(n,shift3, b2_l), c2_l+d2_l+a2_l+b2_l+ancilla2_l)
  b2_l = classical_n_rotateBy_d(b2_l,n,shift3)

  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)


  print('column 3')
  qc.append(one_arx(n,shift0, d3_l), a3_l+b3_l+c3_l+d3_l+ancilla3_l)            # quarter round on column 3
  d3_l = classical_n_rotateBy_d(d3_l,n,shift0)
  qc.append(one_arx(n,shift1, b3_l), c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  b3_l = classical_n_rotateBy_d(b3_l,n,shift1)
  qc.append(one_arx(n,shift2, d3_l), a3_l+b3_l+c3_l+d3_l+ancilla3_l)
  d3_l = classical_n_rotateBy_d(d3_l,n,shift2)
  qc.append(one_arx(n,shift3, b3_l), c3_l+d3_l+a3_l+b3_l+ancilla3_l)
  b3_l = classical_n_rotateBy_d(b3_l,n,shift3)
  
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  print(i, 'diagonal round')
  print('diagonal 0')
  qc.append(one_arx(n,shift0, d3_l), a0_l+b1_l+c2_l+d3_l+ancilla0_l)            # a0,b1,c2,d3    #quarter round on diagonal 0
  print("diag 0 here:",d3_l)
  d3_l = classical_n_rotateBy_d(d3_l,n,shift0)
  print("diag 0 here:",d3_l)
  qc.append(one_arx(n,shift1, b1_l), c2_l+d3_l+a0_l+b1_l+ancilla0_l)  
  b1_l = classical_n_rotateBy_d(b1_l,n,shift1)          
  qc.append(one_arx(n,shift2, d3_l), a0_l+b1_l+c2_l+d3_l+ancilla0_l)  
  d3_l = classical_n_rotateBy_d(d3_l,n,shift2)         
  qc.append(one_arx(n,shift3, b1_l), c2_l+d3_l+a0_l+b1_l+ancilla0_l)  
  b1_l = classical_n_rotateBy_d(b1_l,n,shift3)      

  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)    


  print('diagonal 1')      
  qc.append(one_arx(n,shift0, d0_l), a1_l+b2_l+c3_l+d0_l+ancilla1_l)            # a1,b2,c3,d0    #quarter round on diagonal 1
  d0_l = classical_n_rotateBy_d(d0_l,n,shift0)
  qc.append(one_arx(n,shift1, b2_l), c3_l+d0_l+a1_l+b2_l+ancilla1_l) 
  b2_l = classical_n_rotateBy_d(b2_l,n,shift1)
  qc.append(one_arx(n,shift2, d0_l), a1_l+b2_l+c3_l+d0_l+ancilla1_l)    
  d0_l = classical_n_rotateBy_d(d0_l,n,shift2)
  qc.append(one_arx(n,shift3, b2_l), c3_l+d0_l+a1_l+b2_l+ancilla1_l)  
  b2_l = classical_n_rotateBy_d(b2_l,n,shift3)

  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  print('diagonal 2')
  qc.append(one_arx(n,shift0, d1_l), a2_l+b3_l+c0_l+d1_l+ancilla2_l)            # a2,b3,c0,d1    #quarter round on diagonal 2
  d1_l = classical_n_rotateBy_d(d1_l,n,shift0)
  qc.append(one_arx(n,shift1, b3_l), c0_l+d1_l+a2_l+b3_l+ancilla2_l)  
  b3_l = classical_n_rotateBy_d(b3_l,n,shift1)
  qc.append(one_arx(n,shift2, d1_l), a2_l+b3_l+c0_l+d1_l+ancilla2_l)   
  d1_l = classical_n_rotateBy_d(d1_l,n,shift2)
  qc.append(one_arx(n,shift3, b3_l), c0_l+d1_l+a2_l+b3_l+ancilla2_l)    
  b3_l = classical_n_rotateBy_d(b3_l,n,shift3)

  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  print('diagonal 3')
  qc.append(one_arx(n,shift0, d2_l), a3_l+b0_l+c1_l+d2_l+ancilla3_l)            # a3,b0,c1,d2   #quarter round on diagonal 3
  d2_l = classical_n_rotateBy_d(d2_l,n,shift0)
  qc.append(one_arx(n,shift1, b0_l), c1_l+d2_l+a3_l+b0_l+ancilla3_l)    
  b0_l = classical_n_rotateBy_d(b0_l,n,shift1)
  qc.append(one_arx(n,shift2, d2_l), a3_l+b0_l+c1_l+d2_l+ancilla3_l)     
  d2_l = classical_n_rotateBy_d(d2_l,n,shift2)
  qc.append(one_arx(n,shift3, b0_l), c1_l+d2_l+a3_l+b0_l+ancilla3_l) 
  b0_l = classical_n_rotateBy_d(b0_l,n,shift3)


  # print("Round", i)
  # qc2=qc.copy()
  # qc2.measure_all()
  # measure(qc2,n)

  # break
          

{'10111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100101011101110010111100101011000010111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100101011101110010111100101011': 1}
[0, 1, 2, 3, 4, 5, 6, 7] [8, 9, 10, 11, 12, 13, 14, 15] [16, 17, 18, 19, 20, 21, 22, 23] [24, 25, 26, 27, 28, 29, 30, 31]
[32, 33, 34, 35, 36, 37, 38, 39] [40, 41, 42, 43, 44, 45, 46, 47] [48, 49, 50, 51, 52, 53, 54, 55] [56, 57, 58, 59, 60, 61, 62, 63]
[64, 65, 66, 67, 68, 69, 70, 71] [72, 73, 74, 75, 76, 77, 78, 79] [80, 81, 82, 83, 84, 85, 86, 87] [88, 89, 90, 91, 92, 93, 94, 95]
[96, 97, 98, 99, 100, 101, 102, 103] [104, 105, 106, 107, 108, 109, 110, 111] [112, 113, 114, 115, 116, 117, 118, 119] [120, 121, 122, 123, 124, 125, 126, 127]

0x2b 0x2f 0x77 0xa5
0x75 0x42 0x8 0xe7
0xff 0x5f 0xe3 0xaf
0x15 0x3c 0xaf 0xbf

00101011 00101111 01110111 10100101
01110101 01000010 00001000 11100111
11111111 01011111 11100011 101011

In [ ]:
my_num="0001010100011100011110000"
# my_num="0 0010 1010 0011 1000 1111 0000"
print(len(my_num))
print(get_num(my_num[::-1], [20, 21, 22, 23, 24]))
print(get_num(my_num[::-1], [22, 23, 24, 20, 21]))

25
00010
10000


In [ ]:
# qc.decompose().draw(output='mpl', scale=0.5)

In [ ]:
# qc.draw(filename='abcd')

In [ ]:
# qc.draw(filename='output')

In [ ]:
qc2=qc.copy()
qc2.measure_all()
measure(qc2,n)

{'10111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100101011101110010111100101011000010000011001101110001010110011101011001011110011111101001101111011010110001010011001011100100011100100001111111111011000011001100': 1}
[0, 1, 2, 3, 4, 5, 6, 7] [8, 9, 10, 11, 12, 13, 14, 15] [16, 17, 18, 19, 20, 21, 22, 23] [24, 25, 26, 27, 28, 29, 30, 31]
[32, 33, 34, 35, 36, 37, 38, 39] [40, 41, 42, 43, 44, 45, 46, 47] [48, 49, 50, 51, 52, 53, 54, 55] [56, 57, 58, 59, 60, 61, 62, 63]
[64, 65, 66, 67, 68, 69, 70, 71] [72, 73, 74, 75, 76, 77, 78, 79] [80, 81, 82, 83, 84, 85, 86, 87] [88, 89, 90, 91, 92, 93, 94, 95]
[96, 97, 98, 99, 100, 101, 102, 103] [104, 105, 106, 107, 108, 109, 110, 111] [112, 113, 114, 115, 116, 117, 118, 119] [120, 121, 122, 123, 124, 125, 126, 127]

0xcc 0xb0 0xff 0x21
0x47 0x2e 0x53 0xac
0xbd 0xe9 0xe7 0x65
0x9d 0x15 0x37 0x83

11001100 10110000 11111111 00100001
01000111 00101110 01010011 10101100
10111101 11101001 11100111 01100

#### last addition

ancilla can be increased to reduce depth

In [ ]:
qc.append(additionMod_2n(n).to_gate(), ori_a0_l+a0_l+ancilla0_l)
qc.append(additionMod_2n(n).to_gate(), ori_a1_l+a1_l+ancilla0_l)
qc.append(additionMod_2n(n).to_gate(), ori_a2_l+a2_l+ancilla0_l)
qc.append(additionMod_2n(n).to_gate(), ori_a3_l+a3_l+ancilla0_l)

qc.append(additionMod_2n(n).to_gate(), ori_b0_l+b0_l+ancilla1_l)
qc.append(additionMod_2n(n).to_gate(), ori_b1_l+b1_l+ancilla1_l)
qc.append(additionMod_2n(n).to_gate(), ori_b2_l+b2_l+ancilla1_l)
qc.append(additionMod_2n(n).to_gate(), ori_b3_l+b3_l+ancilla1_l)

qc.append(additionMod_2n(n).to_gate(), ori_c0_l+c0_l+ancilla2_l)
qc.append(additionMod_2n(n).to_gate(), ori_c1_l+c1_l+ancilla2_l)
qc.append(additionMod_2n(n).to_gate(), ori_c2_l+c2_l+ancilla2_l)
qc.append(additionMod_2n(n).to_gate(), ori_c3_l+c3_l+ancilla2_l)

qc.append(additionMod_2n(n).to_gate(), ori_d0_l+d0_l+ancilla3_l)
qc.append(additionMod_2n(n).to_gate(), ori_d1_l+d1_l+ancilla3_l)
qc.append(additionMod_2n(n).to_gate(), ori_d2_l+d2_l+ancilla3_l)
qc.append(additionMod_2n(n).to_gate(), ori_d3_l+d3_l+ancilla3_l)

In [ ]:
qc2=qc.copy()
qc2.measure_all()
measure(qc2,n)

{'10111111101011110011110000010101101011111110001101011111111111111110011100001000010000100111010110100101011101110010111100101011000001000010111001100101000110110010000101001100101001001000101111001001001101011011011100001011110011000110011101101101111111110111': 1}
[0, 1, 2, 3, 4, 5, 6, 7] [8, 9, 10, 11, 12, 13, 14, 15] [16, 17, 18, 19, 20, 21, 22, 23] [24, 25, 26, 27, 28, 29, 30, 31]
[32, 33, 34, 35, 36, 37, 38, 39] [40, 41, 42, 43, 44, 45, 46, 47] [48, 49, 50, 51, 52, 53, 54, 55] [56, 57, 58, 59, 60, 61, 62, 63]
[64, 65, 66, 67, 68, 69, 70, 71] [72, 73, 74, 75, 76, 77, 78, 79] [80, 81, 82, 83, 84, 85, 86, 87] [88, 89, 90, 91, 92, 93, 94, 95]
[96, 97, 98, 99, 100, 101, 102, 103] [104, 105, 106, 107, 108, 109, 110, 111] [112, 113, 114, 115, 116, 117, 118, 119] [120, 121, 122, 123, 124, 125, 126, 127]

0xf7 0xdf 0x76 0xc6
0xbc 0x70 0x5b 0x93
0xbc 0x48 0xca 0x14
0xb2 0x51 0xe6 0x42

11110111 11011111 01110110 11000110
10111100 01110000 01011011 10010011
10111100 01001000 11001010 00010